# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [17]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

True

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

**Mise à jour 29/07**
- Utiliser Supabase, pas ElephantSQL
- fonction generate_ecommerce_data, ne pas utiliser, importer les données du dataset .csv

In [18]:
# Configuration de connexion (à adapter selon votre provider)
DATABASE_CONFIG = {
    'host': os.getenv('DB_HOST'),
    'database': os.getenv('DB_NAME'),
    'user': os.getenv('DB_USER'),
    'password': os.getenv('DB_PASS'),
    'port': os.getenv('DB_PORT')
}

In [19]:
# Création de l'engine SQLAlchemy
# from sqlalchemy.engine import URL puis URL.create ?
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        connection = psycopg2.connect(**DATABASE_CONFIG)
        cursor = connection.cursor()

        cursor.execute('''
            SELECT version();
        ''')

        print(cursor.fetchone())

        connection.commit()
        cursor.close()
        connection.close()

    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True

test_connection()

('PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 13.2.0, 64-bit',)


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [20]:
df_sellers = pd.read_csv('data/olist_sellers_dataset.csv')
df_products = pd.read_csv('data/olist_products_dataset.csv')
df_customers = pd.read_csv('data/olist_customers_dataset.csv')
df_orders = pd.read_csv('data/olist_orders_dataset.csv')
df_order_items = pd.read_csv('data/olist_order_items_dataset.csv')

df_sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [21]:
order_status_enum_list = df_orders['order_status'].unique()
print(f'values: {order_status_enum_list}')
print(f'type: {type(order_status_enum_list)}\n')

# on récupère les éléments sous la forme de list et on s'assure de ne pas avoir de nan
order_status_enum_list = [i for i in order_status_enum_list if pd.notna(i)]
print(f'values: {order_status_enum_list}')
print(f'type: {type(order_status_enum_list)}')

values: ['delivered' 'invoiced' 'shipped' 'processing' 'unavailable' 'canceled'
 'created' 'approved']
type: <class 'numpy.ndarray'>

values: ['delivered', 'invoiced', 'shipped', 'processing', 'unavailable', 'canceled', 'created', 'approved']
type: <class 'list'>


In [22]:
product_category_enum_list = df_products['product_category_name'].unique()
print(f'values: {product_category_enum_list}')
print(f'type: {type(product_category_enum_list)}\n')

product_category_enum_list = [i for i in product_category_enum_list if pd.notna(i)]
print(f'values: {product_category_enum_list}')
print(f'type: {type(product_category_enum_list)}\n')

values: ['perfumaria' 'artes' 'esporte_lazer' 'bebes' 'utilidades_domesticas'
 'instrumentos_musicais' 'cool_stuff' 'moveis_decoracao'
 'eletrodomesticos' 'brinquedos' 'cama_mesa_banho'
 'construcao_ferramentas_seguranca' 'informatica_acessorios'
 'beleza_saude' 'malas_acessorios' 'ferramentas_jardim'
 'moveis_escritorio' 'automotivo' 'eletronicos' 'fashion_calcados'
 'telefonia' 'papelaria' 'fashion_bolsas_e_acessorios' 'pcs'
 'casa_construcao' 'relogios_presentes'
 'construcao_ferramentas_construcao' 'pet_shop' 'eletroportateis'
 'agro_industria_e_comercio' nan 'moveis_sala' 'sinalizacao_e_seguranca'
 'climatizacao' 'consoles_games' 'livros_interesse_geral'
 'construcao_ferramentas_ferramentas' 'fashion_underwear_e_moda_praia'
 'fashion_roupa_masculina'
 'moveis_cozinha_area_de_servico_jantar_e_jardim'
 'industria_comercio_e_negocios' 'telefonia_fixa'
 'construcao_ferramentas_iluminacao' 'livros_tecnicos'
 'eletrodomesticos_2' 'artigos_de_festas' 'bebidas' 'market_place'
 'la_cuisine

In [23]:
### 🗃️ Création des tables SQL 
# Complétez pour les autres tables
# N'oubliez pas les contraintes de clés étrangères !

def create_tables():
    """
    Créez les tables dans PostgreSQL

    Tips :
    - Utilisez des SERIAL pour les IDs auto-increment
    - Ajoutez des index sur les clés étrangères
    - Incluez des contraintes de validation
    """

    create_sellers = '''
        CREATE TABLE IF NOT EXISTS sellers (
            seller_id VARCHAR(50) PRIMARY KEY,
            seller_city VARCHAR(100),
            seller_state VARCHAR(2)
        )
    '''

    create_products = f'''
        CREATE TYPE product_category_enum as ENUM (
            '{"', '".join(product_category_enum_list)}'
        );

        CREATE TABLE IF NOT EXISTS products (
            product_id VARCHAR(50) PRIMARY KEY,
            product_category product_category_enum,
            product_weight_g DECIMAL(8,1)
        )
    '''

    create_customers = """
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR(50) PRIMARY KEY,
        customer_unique_id VARCHAR(50) UNIQUE,
        customer_city VARCHAR(100),
        customer_state VARCHAR(2),
        customer_zip_code VARCHAR(10),
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """

    create_orders = f'''
        CREATE TYPE order_status_enum AS ENUM (
            '{"', '".join(order_status_enum_list)}'
        );

        CREATE TABLE IF NOT EXISTS orders (
            order_id VARCHAR(50) PRIMARY KEY,
            customer_id VARCHAR(50) references customers(customer_id),
            order_status order_status_enum NOT NULL,
            order_date TIMESTAMP,
            order_delivered_date TIMESTAMP
        )
    '''
    create_order_items = '''
        CREATE TABLE IF NOT EXISTS order_items (
            order_id VARCHAR(50) references orders(order_id),
            product_id VARCHAR(50) references products(product_id),
            seller_id VARCHAR(50) references sellers(seller_id),
            price DECIMAL(8,2),
            freight_value DECIMAL(8,2)
        )
    '''
    
    try:
        with engine.connect() as conn:
            conn.execute(text(create_sellers))
            conn.execute(text(create_products))
            conn.execute(text(create_customers))
            conn.execute(text(create_orders))
            conn.execute(text(create_order_items))

            conn.commit()
    except Exception as e:
        print(f'error: {e}')

# create_tables()

In [24]:
def alter_tables():
    alter_customers = '''
        ALTER TABLE customers
            DROP COLUMN customer_unique_id, DROP COLUMN customer_zip_code, DROP COLUMN created_at
    '''
    try:
        with engine.connect() as conn:
            conn.execute(text(alter_customers))
            
            conn.commit()
    except Exception as e:
        print(f'error: {e}')

# alter_tables()

### Traitement des données

In [25]:
# drop unused columns
df_sellers = df_sellers.drop('seller_zip_code_prefix', axis='columns')
df_products = df_products.drop(['product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_length_cm', 'product_height_cm', 'product_width_cm'], axis=1)
df_customers = df_customers.drop(['customer_unique_id', 'customer_zip_code_prefix'], axis=1)
df_orders = df_orders.drop(['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date'], axis=1)
df_order_items = df_order_items.drop(['order_item_id', 'shipping_limit_date'], axis=1)

df_customers.head()

,customer_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,campinas,SP


In [26]:
# rename columns
df_products = df_products.rename({'product_category_name':'product_category'}, axis=1)
df_orders = df_orders.rename({'order_purchase_timestamp':'order_date', 'order_delivered_customer_date':'order_delivered_date'}, axis=1)

df_order_items.head()

,order_id,product_id,seller_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14


In [27]:
df_list = [df_sellers, df_products, df_customers, df_orders, df_order_items]

def check_types_in_df_list(df_list: list):
    for i in df_list:
        print(f'{i.dtypes}\n')

check_types_in_df_list(df_list)

seller_id       object
seller_city     object
seller_state    object
dtype: object

product_id           object
product_category     object
product_weight_g    float64
dtype: object

customer_id       object
customer_city     object
customer_state    object
dtype: object

order_id                object
customer_id             object
order_status            object
order_date              object
order_delivered_date    object
dtype: object

order_id          object
product_id        object
seller_id         object
price            float64
freight_value    float64
dtype: object



In [28]:
def check_na(df_list: list):
    for df in df_list:
        print(f'{df.isna().sum()}\n')

check_na(df_list)

seller_id       0
seller_city     0
seller_state    0
dtype: int64

product_id            0
product_category    610
product_weight_g      2
dtype: int64

customer_id       0
customer_city     0
customer_state    0
dtype: int64

order_id                   0
customer_id                0
order_status               0
order_date                 0
order_delivered_date    2965
dtype: int64

order_id         0
product_id       0
seller_id        0
price            0
freight_value    0
dtype: int64



In [29]:
def fill_db(df: pd.DataFrame, table_name: str):
    df.to_sql(table_name, if_exists='append', con=engine.connect(), index=False)

# fill_db(df_sellers, 'sellers')
# fill_db(df_products, 'products')
# fill_db(df_customers, 'customers')
# fill_db(df_orders, 'orders')
# fill_db(df_order_items, 'order_items')

## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)

-- Créez les segments RFM (Champions, Loyaux, À risque, etc.)
SELECT
    customer_id,
    customer_state,
    recency_score,
    frequency_score,
    monetary_score,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        -- Ajoutez les autres segments
        ELSE 'Others'
    END as customer_segment
FROM customer_metrics;
```

In [ ]:
# RFM
rfm = '''
    -- on crée une CTE (common table expression) : une table temporaire qu'on peut utiliser pour une autre requête
    WITH customer_metrics AS (
        SELECT
            c.customer_id,
            c.customer_state,

            -- RECENCY
            CURRENT_DATE - MAX(o.order_date)::date AS days_since_last_order,
            -- on divise en 5 groupes en se basant sur l'order le plus récent de chaque customer
            NTILE(5) OVER (
                ORDER BY MAX(o.order_date)
            ) AS recency_score,

            -- FREQUENCY
            COUNT(DISTINCT o.order_id) AS nb_of_orders,
            NTILE(5) OVER (
                ORDER BY COUNT(o.order_id)            
            ) AS frequency_score,

            -- MONETARY
            SUM(oi.price + oi.freight_value) AS total_spent,
            NTILE(5) OVER (
                ORDER BY SUM(oi.price + oi.freight_value) 
            ) AS monetary_score

        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        JOIN order_items oi ON o.order_id = oi.order_id
        WHERE o.order_status = 'delivered'
        GROUP BY c.customer_id, c.customer_state
    )

    SELECT
        *,
        CASE
            WHEN recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4 THEN 'Champions'
            WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
            WHEN recency_score >= 4 AND frequency_score >= 2 AND frequency_score <= 3 THEN 'Potential Loyalists'
            WHEN recency_score >= 2 AND recency_score <= 3 AND frequency_score >= 3 THEN 'At Risk'
            WHEN recency_score <= 2 AND frequency_score <= 2 THEN 'Hibernating'
            WHEN recency_score = 1 THEN 'Lost'
            ELSE 'Others'
        END AS customer_segment
    FROM customer_metrics
'''

df_result = pd.read_sql(rfm, engine)
df_result.head(10)

,customer_id,customer_state,days_since_last_order,recency_score,nb_of_orders,frequency_score,total_spent,monetary_score,customer_segment
0,86dc2ffce2dfff336de2f386a786e574,SP,3242,1,1,5,143.46,4,Lost
1,355077684019f7f60a031656bd7262b8,SP,3224,1,1,1,45.46,1,Hibernating
2,7ec40b22510fdbea1b08921dd39e63d8,RS,3224,1,1,1,39.09,1,Hibernating
3,6f989332712d3222b6571b1cf5b835ce,RS,3224,1,1,2,53.73,1,Hibernating
4,b8cf418e97ae795672d326288dfab7a7,SP,3224,1,1,3,133.46,4,Lost
5,7812fcebfc5e8065d31e1bb5f0017dae,SP,3224,1,1,1,40.95,1,Hibernating
6,e6f959bf384d1d53b6d68826699bba12,GO,3224,1,1,3,154.57,4,Lost
7,dc607dc98d6a11d5d04d9f2a70aa6c34,MG,3224,1,1,4,92.27,3,Lost
8,4f3f778022aefa22b9f9e52d2c47edf3,SP,3223,1,1,2,65.50,2,Hibernating
9,b3a9bf200375f53cc5c6991919c356fd,SP,3223,1,1,1,44.23,1,Hibernating


In [86]:
print(f'{df_result['recency_score'].value_counts()}\n')
print(f'{df_result['frequency_score'].value_counts()}\n')
print(f'{df_result['monetary_score'].value_counts()}')

recency_score
1    19296
2    19296
3    19296
4    19295
5    19295
Name: count, dtype: int64

frequency_score
1    19296
2    19296
3    19296
5    19295
4    19295
Name: count, dtype: int64

monetary_score
1    19296
3    19296
2    19296
4    19295
5    19295
Name: count, dtype: int64


In [153]:
df_result['customer_segment'].value_counts()

customer_segment
Loyal Customers        27183
Hibernating            15772
Others                 15366
At Risk                11443
Lost                   11377
Champions               7883
Potential Loyalists     7454
Name: count, dtype: int64

In [168]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur (ne pas faire)
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen

    WITH top_states AS (
        SELECT
            c.customer_state,
            SUM(oi.price) AS total_revenue,
            COUNT(DISTINCT o.order_id) AS nb_of_orders,
            SUM(oi.price) / COUNT(DISTINCT o.order_id) AS average_order_price -- formule pour avoir la moyenne de la somme des oi.price en fonction des o.order_id
        FROM customers c
        JOIN orders o ON c.customer_id = o.customer_id
        JOIN order_items oi ON o.order_id = oi.order_id
        GROUP BY c.customer_state
        ORDER BY total_revenue DESC 
        LIMIT 10
    )
    
    SELECT * FROM top_states
    """

    query_mom = '''
        WITH monthly_revenue_by_state AS (
            SELECT 
                c.customer_state,
                TO_CHAR(o.order_date, 'YYYY-MM') as month,
                SUM(oi.price) AS revenue
            FROM customers c 
            JOIN orders o ON c.customer_id = o.customer_id
            JOIN order_items oi ON o.order_id = oi.order_id
            GROUP BY c.customer_state, month
            
        )

        SELECT 
            *,
            LAG(revenue) OVER (
                -- on fait en sorte que le lag ne se fasse que dans la partition customer_state
                PARTITION BY customer_state
                ORDER BY month
            ) AS prev_month_revenue,
            revenue - LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month) AS mom_comparison,
            -- utilisation de case pour gérer le cas où on ferait face à une division par zero
            CASE 
                WHEN LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month) IS NOT NULL AND LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month) != 0
                THEN ROUND(
                (
                    (revenue - LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month)) * 100 / LAG(revenue) OVER (PARTITION BY customer_state ORDER BY month)
                )::numeric, 
                2
                )
            END AS mom_percentage
        FROM monthly_revenue_by_state
        ORDER BY customer_state, month DESC
    '''

    query_conversion = '''
        WITH conversion_rate AS (
            SELECT
                c.customer_city,
                COUNT(o.order_id) AS nb_of_orders,
                COUNT(DISTINCT c.customer_id) AS nb_of_customers,
                COUNT(DISTINCT CASE WHEN o.order_id IS NOT NULL THEN c.customer_id END) AS nb_of_customers_with_orders,
                ROUND(
                    COUNT(DISTINCT CASE WHEN o.order_id IS NOT NULL THEN c.customer_id END) * 100 / COUNT(DISTINCT c.customer_id),
                    2
                ) AS conversion_rate_percentage
            FROM customers c
            LEFT JOIN orders o ON c.customer_id = o.customer_id
            GROUP BY c.customer_city
        )

        SELECT * FROM conversion_rate
    '''

    top_states = pd.read_sql(query_top_states, engine)
    print('top 10 states by revenue')
    display(top_states)

    mom = pd.read_sql(query_mom, engine)
    print('month on month')
    display(mom)

    conversion_rate = pd.read_sql(query_conversion, engine)
    print('conversion rate')
    display(conversion_rate)

geographic_sales_analysis()

top 10 states by revenue


,customer_state,total_revenue,nb_of_orders,average_order_price
0,SP,5202955.05,41375,125.751179
1,RJ,1824092.67,12762,142.931568
2,MG,1585308.03,11544,137.327445
3,RS,750304.02,5432,138.126661
4,PR,683083.76,4998,136.671421
5,SC,520553.34,3612,144.117757
6,BA,511349.99,3358,152.278139
7,DF,302603.94,2125,142.401854
8,GO,294591.95,2007,146.782237
9,ES,275037.31,2025,135.820894


month on month


,customer_state,month,revenue,prev_month_revenue,mom_comparison,mom_percentage
0,AC,2018-08,446.69,1469.97,-1023.28,-69.61
1,AC,2018-07,1469.97,427.90,1042.07,243.53
2,AC,2018-06,427.90,1364.89,-936.99,-68.65
3,AC,2018-05,1364.89,422.98,941.91,222.68
4,AC,2018-04,422.98,97.80,325.18,332.49
...,...,...,...,...,...,...
554,TO,2017-05,2124.33,3192.84,-1068.51,-33.47
555,TO,2017-04,3192.84,727.66,2465.18,338.78
556,TO,2017-03,727.66,765.60,-37.94,-4.96
557,TO,2017-02,765.60,629.89,135.71,21.55


conversion rate


,customer_city,nb_of_orders,nb_of_customers,nb_of_customers_with_orders,conversion_rate_percentage
0,abadia dos dourados,3,3,3,100.0
1,abadiania,1,1,1,100.0
2,abaete,12,12,12,100.0
3,abaetetuba,11,11,11,100.0
4,abaiara,2,2,2,100.0
...,...,...,...,...,...
4114,xinguara,9,9,9,100.0
4115,xique-xique,3,3,3,100.0
4116,zacarias,2,2,2,100.0
4117,ze doca,5,5,5,100.0


#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

In [170]:
query_patterns_saisonniers = '''
    SELECT
        EXTRACT(YEAR FROM order_date) as year,
        EXTRACT(MONTH FROM order_date) as month,
        EXTRACT(DOW FROM order_date) as day_of_week,
        COUNT(*) as order_count,
        SUM(price + freight_value) as total_revenue,
        AVG(price + freight_value) as avg_order_value
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE order_status = 'delivered'
    GROUP BY ROLLUP(
        EXTRACT(YEAR FROM order_date),
        EXTRACT(MONTH FROM order_date),
        EXTRACT(DOW FROM order_date)
    )
ORDER BY year, month, day_of_week;
'''

patterns_saisonniers = pd.read_sql(query_patterns_saisonniers, engine)
display(patterns_saisonniers)

,year,month,day_of_week,order_count,total_revenue,avg_order_value
0,2016.0,9.0,4.0,3,143.46,47.820000
1,2016.0,9.0,NaN,3,143.46,47.820000
2,2016.0,10.0,0.0,26,2904.19,111.699615
3,2016.0,10.0,1.0,44,4456.26,101.278636
4,2016.0,10.0,2.0,63,9821.42,155.895556
...,...,...,...,...,...,...
171,2018.0,8.0,5.0,1003,135803.41,135.397218
172,2018.0,8.0,6.0,758,110963.05,146.389248
173,2018.0,8.0,NaN,7142,985491.64,137.985388
174,2018.0,NaN,NaN,60324,8451584.77,140.103189


## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL